In [1]:
import sys, cv2, math, time
import numpy as np
from board_detection import detect_board
import classes, custom_model

font = cv2.FONT_HERSHEY_SIMPLEX

Using TensorFlow backend.


In [2]:
def draw_line(p0,p1,img,color=(0,255,0)):
    line_thickness=5
    p0=tuple(p0)
    p1=tuple(p1)
    cv2.line(img=img,
             pt1=(int(p0[0]),int(p0[1])),
             pt2=(int(p1[0]),int(p1[1])),
             color=color, thickness=line_thickness, lineType=8)

In [3]:
def main():

    # Initialize image capture from camera.
    video_capture = cv2.VideoCapture('../data/proto/full.mp4') # Open video capture object

    # Check if camera opened successfully
    if (video_capture.isOpened()== False):
        print("Error opening video stream or file")
        sys.exit()

    model=custom_model.custom_model()
    model.load_model()
        
    #output file
    out_name='../data/proto/output_video'
    ext='.mp4'
    out_string=out_name+ext
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    print("saving output to %s" % out_string)

    out = cv2.VideoWriter(out_string,fourcc, 24, (1440,1440))

    frame_count=0
    det_count=0
    do_frames=20
    
    while (video_capture.isOpened()):
#    for i in range(50):
#        print("Frame # "+str(frame_count))
#         if frame_count>=do_frames: break
        ret, bgr_image = video_capture.read()
        bgr_image = cv2.rotate(bgr_image, cv2.ROTATE_90_CLOCKWISE)
        if ret:
            board=classes.Board()
            print(frame_count)
            frame_height = bgr_image.shape[0]
            frame_width = bgr_image.shape[1]

            try:
                ortho_image,intersections=detect_board(bgr_image,do_draw_board=True)
                intersections=np.reshape(np.transpose(np.reshape(intersections,newshape=(9,9,3)),(1,0,2)),newshape=(81,3))
                size=ortho_image.shape[0]

                #populate board accordingly 
                square_size=int(size/8)
                for i in range(8):
                    for j in range(8):
                        idx=j*8+i
                        crop_img = ortho_image[j*square_size:j*square_size+square_size, i*square_size:i*square_size+square_size]

                        free_score=model.run_inference(crop_img)
                        free_score=int(round(free_score))
                        if free_score == 0 : 
                            board.squares[idx].popu=True
                            board.queens.append(classes.Queen(loc_=idx))
                        print(str(free_score)+" ",end="")
                    print()
                is_finished=board.update_safe()
                color=(0,255,0)
                if is_finished: color=(255,0,0)
    #             print(intersections.shape)
    #             print(intersections)
                for q in board.queens:
                    sq=q.loc
                    row=int(sq/8)
                    col=int(sq%8)
                    p00=int(9*row+col)
                    p01=int(9*(row+1)+col)
                    p10=int(9*row+col+1)
                    p11=int(9*(row+1)+col+1)
                    if not q.safe: color=(0,0,255)
                    line_thickness=4
                    draw_line(p0=intersections[p00],p1=intersections[p01],img=bgr_image,color=color)
                    draw_line(p0=intersections[p01],p1=intersections[p11],img=bgr_image,color=color)
                    draw_line(p0=intersections[p11],p1=intersections[p10],img=bgr_image,color=color)
                    draw_line(p0=intersections[p10],p1=intersections[p00],img=bgr_image,color=color)

                bgr_image[0:size, frame_width-size:frame_width] = ortho_image
                det_count+=1
            except:
                cv2.putText(bgr_image,"board not detected",(400,100), font, 1,(255,255,215),2)

            # display frames
            cv2.putText(bgr_image,"%d" % frame_count,(20,100), font, 1,(255,255,215),2)

            cv2.imshow("Input image", bgr_image)

            # write to video on disk
            out.write(bgr_image)

            # Show images and wait for xx msec (0 = wait till keypress).
            key_pressed = cv2.waitKey(1) & 0xFF
            if key_pressed == 27 or key_pressed == ord('q'):
                break # Quit on ESC or q
        else: break
        frame_count+=1
    print("Board detected "+str(det_count)+" frames out of "+str(frame_count)+".")
    print("Ratio: "+str(det_count/frame_count))
    #finished do cleanup
    video_capture.release()
    out.release()
    cv2.destroyAllWindows()
    return intersections

In [4]:
t1=time.time()
intersections=main()
t2=time.time()
print("elapsed "+str(t2-t1))

saving output to ../data/proto/output_video.mp4
0
1 1 1 1 1 1 1 1 
0 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 0 
1 1 0 1 1 1 1 1 
1
1 1 1 1 1 1 1 1 
0 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 0 
1 1 0 1 1 1 1 1 
2
1 1 1 1 1 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 0 
1 1 0 1 1 1 1 1 
3
1 1 1 1 1 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 0 
1 1 0 1 1 1 1 1 
4
1 1 1 1 1 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 0 
1 1 0 1 1 1 1 1 
5
1 1 1 1 1 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 0 
1 1 0 1 1 1 1 1 
6
1 1 1 1 1 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 0 
1 1

1 1 0 0 1 1 1 0 
1 1 0 1 1 1 1 1 
59
1 1 1 1 1 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 0 0 0 1 1 1 1 
1 1 0 0 1 1 1 0 
1 1 0 1 1 1 1 1 
60
1 1 1 1 1 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 0 0 0 1 1 1 1 
1 1 0 0 1 1 1 0 
1 1 0 1 1 1 1 0 
61
1 1 1 1 1 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 0 0 1 1 1 1 
1 1 0 0 1 1 1 0 
1 1 0 1 1 1 1 1 
62
1 1 1 1 1 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 0 0 1 1 1 1 
1 0 0 0 1 1 1 0 
1 1 0 1 1 1 1 0 
63
1 1 1 1 1 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 0 0 1 
1 0 0 0 1 1 1 1 
1 1 1 0 1 1 1 0 
0 1 0 1 1 1 1 1 
64
1 1 1 1 1 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 0 0 1 1 1 1 
1 0 1 0 1 1 1 0 
1 1 0 1 1 1 1 1 
65
1 1 1 1 1 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 1 0 1 1 1 1 
1 0 0 1 1 1 1 0 
1 1 0 1 1 

1 1 1 1 1 1 1 0 
1 0 0 1 1 1 1 1 
118
1 1 1 0 0 1 1 1 
1 0 0 1 0 1 1 1 
1 1 1 0 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 0 1 1 1 0 1 
1 1 0 0 1 1 1 1 
1 1 1 1 1 1 1 0 
1 0 0 1 1 1 1 1 
119
1 1 1 0 0 1 1 1 
1 0 0 0 0 1 1 1 
1 1 1 0 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 0 1 1 1 0 1 
1 1 0 0 1 1 1 1 
1 1 1 1 1 1 1 0 
1 0 0 1 1 1 1 1 
120
1 1 1 1 0 1 1 1 
1 0 0 1 0 1 1 1 
1 1 1 0 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 0 1 1 1 0 1 
1 1 0 0 1 1 1 1 
1 1 1 1 1 1 1 0 
1 0 0 1 1 1 1 1 
121
1 1 1 0 0 1 1 1 
1 0 0 0 0 1 1 1 
1 1 1 0 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 0 1 1 1 0 1 
1 1 1 0 1 1 1 1 
1 0 1 1 1 1 1 0 
1 0 0 1 1 1 1 1 
122
1 1 1 0 0 1 1 1 
1 0 0 1 1 1 1 1 
1 1 1 0 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 0 1 1 1 0 1 
1 1 1 0 1 1 1 1 
1 0 1 1 1 1 1 0 
1 0 0 1 1 1 1 1 
123
1 1 1 1 0 1 1 1 
1 0 0 1 1 1 1 1 
1 1 0 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 0 1 1 1 0 1 
1 1 1 0 1 1 1 1 
1 0 1 1 1 1 1 0 
1 1 0 1 1 1 1 1 
124
1 1 1 1 0 1 1 1 
1 0 1 1 1 1 1 1 
1 0 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
0 1 1 0 1 1 1 1 
1 0 1 1 1 1 1 0 
1 1

1 1 1 1 1 1 1 0 
1 0 0 1 1 1 1 1 
235
1 1 1 1 0 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 1 0 1 1 1 1 
1 1 0 1 1 1 1 0 
0 0 0 1 1 1 1 1 
236
1 1 1 1 0 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 0 0 1 1 1 1 
1 0 0 0 1 1 1 0 
1 0 1 1 1 1 1 1 
237
1 1 1 1 0 1 1 1 
0 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 0 0 1 1 1 1 
1 0 0 0 1 1 1 0 
1 1 1 1 1 1 1 1 
238
1 1 1 1 0 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 0 0 1 1 1 1 
1 1 0 0 1 1 1 0 
1 1 1 1 1 1 1 1 
239
1 1 1 1 0 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 0 0 1 1 1 1 
1 0 1 0 1 1 1 0 
1 1 1 1 1 1 1 1 
240
1 1 1 1 0 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 0 
1 1 1 1 1 1 1 1 
241
1 1 1 1 0 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 0 
1 1

0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 0 
1 1 0 1 1 1 1 1 
294
1 1 1 1 0 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 0 
1 1 0 1 1 1 1 1 
295
1 1 1 1 0 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 0 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 0 
1 1 0 1 1 1 1 1 
296
1 1 1 1 0 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 0 
1 1 0 1 1 1 1 1 
297
1 1 1 1 0 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 0 
1 1 0 1 1 1 1 1 
298
1 1 1 1 0 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 0 
1 1 0 1 1 1 1 1 
299
1 1 1 1 0 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 0 
1 1 0 1 1 1 1 1 
300
1 1 1 1 0 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1

352
1 1 1 1 0 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 0 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 0 
1 1 0 1 1 1 1 1 
353
1 1 1 1 0 1 1 1 
1 0 1 1 1 1 0 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 0 
1 1 0 1 1 1 1 1 
354
1 1 1 1 0 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 0 
1 1 0 1 1 1 1 1 
355
1 1 1 1 0 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 0 
1 1 0 1 1 1 1 1 
356
1 1 1 1 0 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 0 
1 1 0 1 1 1 1 1 
357
1 1 1 1 0 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 0 
1 1 0 1 1 1 1 1 
358
1 1 1 1 0 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 0 
1 1 0 1 1 1 1 1 
359
1 1 1 1 0 1 1 1 

1 1 1 1 1 1 1 0 
1 1 0 1 0 1 1 1 
411
1 1 1 1 0 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 0 
1 1 0 1 0 1 1 1 
412
1 1 1 1 0 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 0 
1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 0 
1 1 0 1 0 1 1 1 
413
1 1 1 1 0 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 0 
1 1 0 1 0 1 1 1 
414
1 1 1 1 0 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 0 
1 1 0 1 0 1 1 1 
415
1 1 1 1 0 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 0 
1 1 0 1 0 1 1 1 
416
1 1 1 1 0 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 0 
1 1 0 1 0 1 1 1 
417
1 1 1 1 0 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 1 1 1 1 1 1 
1 1 1 1 1 1 1 0 
1 1

1 1 0 1 1 1 1 1 
529
1 1 1 1 0 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 1 0 1 1 1 1 
1 1 1 1 1 1 1 0 
1 1 0 1 1 1 1 1 
530
1 1 1 1 0 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 1 0 1 1 1 1 
1 1 1 1 1 1 1 0 
1 1 0 1 1 1 1 1 
531
1 1 1 1 0 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 1 0 1 1 1 1 
1 1 1 1 1 1 1 0 
1 1 0 1 1 1 1 1 
532
1 1 1 1 0 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 1 0 1 1 1 1 
1 1 1 1 1 1 1 0 
1 1 0 1 1 1 1 1 
533
1 1 1 1 0 1 1 1 
1 0 1 1 1 1 1 1 
1 1 1 1 1 0 1 1 
0 1 1 1 1 1 1 1 
1 1 1 1 1 1 0 1 
1 1 1 0 1 1 1 1 
1 1 1 1 1 1 1 0 
1 1 0 1 1 1 1 1 
534
0 1 0 0 0 1 0 1 
1 0 0 0 0 0 0 0 
0 0 1 0 1 0 1 1 
1 0 0 0 0 0 0 1 
0 0 0 0 0 0 0 0 
1 1 1 0 0 0 1 0 
0 1 0 1 0 1 1 1 
0 0 0 1 1 0 1 1 
535
0 0 0 0 0 1 0 1 
1 0 0 0 0 0 0 0 
0 0 0 0 0 0 1 1 
1 0 0 0 0 0 0 1 
0 0 0 0 0 0 0 0 
1 1 1 0 0 0 1 0 
0 1 1 1 0 1 1 1 
0 0 0 1 1 0 1 1 
536